In [ ]:
sounds = {}
labels = {}
labeled = False
bad = set()
badp = 0
glyph_freq = {}
glyph_phone = {}
with open('phoible/data/upsid-vowels-pruned.csv', 'r') as f:
    for line in f.readlines():
        if not labeled:
            for i, item in enumerate(line.split("\t")):
                labels[i] = item.strip()
            labeled = True
        else:
            items = [i.strip() for i in line.split("\t")]
            language = items[0]
            if language not in sounds:
                sounds[language] = []
            append = True
            phone = {}
            for i, data in enumerate(items):
                if i < 4:
                    continue
                else:
                    if data == '0':
                        phone[labels[i]] = '-'
                    else:
                        phone[labels[i]] = data
                if data == '+,-' or data == '-,+':
                    append = False
                    bad.add(language)
            if append:
                sounds[language].append(phone)
                glyph = items[3]
                if glyph in glyph_freq:
                    glyph_freq[glyph] += 1
                    assert glyph_phone[glyph] == phone
                else:
                    glyph_freq[glyph] = 1
                    glyph_phone[glyph] = phone
            else:
                badp += 1
#                 sounds[labels[i]] = item
langs = sounds

In [ ]:
print(len(langs))
# print(bad)
print(len(bad))
print(badp)
print(sum([len(langs[x]) for x in bad]))
print(686)

In [ ]:
# len(glyph_freq.keys())
# len(glyph_phone.keys())
sorted_freq = sorted(glyph_freq.items(), reverse=True, key=lambda x: x[1])

In [ ]:
sorted_freq[:10]

In [ ]:
bi_features = ['anterior', 'labiodental', 'tone', 'labial', 'tap', 'raisedLarynxEjective', 'loweredLarynxImplosive', 'click', 'spreadGlottis', 'trill', 'lateral', 'distributed', 'fortis', 'syllabic', 'advancedTongueRoot', 'delayedRelease', 'constrictedGlottis', 'continuant', 'nasal', 'sonorant', 'coronal', 'strident', 'dorsal', 'periodicGlottalSource', 'stress', 'short', 'approximant', 'consonantal', 'epilaryngealSource', 'retractedTongueRoot']
main_features = ['back', 'high', 'low', 'front', 'long', 'round']
import random
def get_random_features():
    random.shuffle(bi_features)
    random.shuffle(main_features)
    features = ['tense'] + bi_features[:] + main_features[:]
    return features

In [ ]:
features = get_random_features()
print(features)
print(len(features))

In [ ]:
def test_comp(vowels, feature):
    for a in vowels:
        for b in vowels:
            if a == b:
                continue
            copy_a = a.copy()
            copy_b = b.copy()
            del copy_a[feature]
            del copy_b[feature]
            if copy_a == copy_b:
#                 for v in vowels:
#                     del v[feature]
                return True
    for v in vowels:
        del v[feature]
    return False

def prune(vowels):
    vowels_copy = [d.copy() for d in vowels]
    keep = [f for f in get_random_features() if test_comp(vowels_copy, f) ]
    return keep

In [ ]:
langs_min_features = {}
import math
def get_best_prune(lang_sounds):
    limit = 10
    counter = 0
    
    theo_min = int(math.ceil(math.log(len(lang_sounds), 2)))
    
    best = prune(lang_sounds)
    while counter < limit:
        pruned = prune(lang_sounds)
        if len(pruned) < theo_min:
            print(lang)
            print(len(lang_sounds))
            print(pruned)
            raise
        elif len(pruned) == theo_min:
            best = pruned
            break
        elif len(best) > len(pruned):
            best = pruned
        counter += 1
    return best
    
for lang in langs.keys():
    langs_min_features[lang] = get_best_prune(langs[lang])


In [ ]:
langs_pruned = {}
for lang in langs:
    langs_pruned[lang] = []
    features = langs_min_features[lang]
    for item in langs[lang]:
        new_features = {key: item[key] for key in features}
        assert len(new_features) == len(features)
        langs_pruned[lang].append(new_features)

In [ ]:
sound_bank = []
for sound in glyph_freq:
    sound_bank.extend(glyph_freq[sound] * [sound])

In [ ]:
# import json # for debugging
samples = {}
min_count = 3 #3
max_count = 16 #20
num_each = 500 #100
for size in range(min_count, max_count + 1):
    print('currently on %s', size)
    samples[size] = []
    for i in range(num_each):
        sample = []
        while len(sample) < size:
            sound = random.choice(sound_bank)
            if glyph_phone[sound] not in sample:
                sample.append(glyph_phone[sound])
#         used for debugging
#         set_check = set([(json.dumps(thingy, sort_keys=True)) for thingy in sample])
#         if len(set_check) != size:
#             print size
#             print len(set_check)
#             print sample
#             raise
        # now we have a sample
        min_features = get_best_prune(sample)
        new_sounds = []
        for item in sample:
            new_features = {key: item[key] for key in min_features}
            assert len(new_features) == len(min_features)
            new_sounds.append(new_features)
        samples[size].append(new_sounds)

In [ ]:
uni_glyphs = glyph_freq.keys()
# import json # for debugging
uniform_samples = {}
min_count = 3 #3
max_count = 16 #16
num_each = 100 #100
for size in range(min_count, max_count + 1):
    print('currently on %s', size)
    uniform_samples[size] = []
    for i in range(num_each):
        sample = []
        while len(sample) < size:
            sound = random.choice(uni_glyphs)
            if glyph_phone[sound] not in sample:
                sample.append(glyph_phone[sound])
        min_features = get_best_prune(sample)
        new_sounds = []
        for item in sample:
            new_features = {key: item[key] for key in min_features}
            assert len(new_features) == len(min_features)
            new_sounds.append(new_features)
        uniform_samples[size].append(new_sounds)

In [ ]:
from numpy import percentile

In [ ]:
econs = {}
def get_econ(sounds):
    num_sounds = len(sounds)
    total_combo = 2 ** len(sounds[0])
    first = sounds[0]
    if 'high' in first and 'low' in first:
        total_combo *= 3.0/4
    if 'front' in first and 'back' in first:
        total_combo *= 3.0/4
    return 1.0 * num_sounds / total_combo
for lang in langs_pruned:
    econs[lang] = get_econ(langs_pruned[lang])

In [ ]:
econ_by_value = {}
for l in langs_pruned:
    sounds = langs_pruned[l]
    number = len(sounds)
    if number not in econ_by_value:
        econ_by_value[number] = []
    econ = get_econ(sounds)
    econ_by_value[number].append(econ)

econ_samples = {}
for size in range(min_count, max_count + 1):
    econ_samples[size] = []
    for inv in samples[size]:
        econ = get_econ(inv)
        econ_samples[size].append(econ)

econ_samples_uniform = {}
for size in range(min_count, max_count + 1):
    econ_samples_uniform[size] = []
    for inv in uniform_samples[size]:
        econ = get_econ(inv)
        econ_samples_uniform[size].append(econ)

In [ ]:
inventories = [econ_by_value, econ_samples, econ_samples_uniform]
for inv in inventories:
    print("Next")
    for size in range(min_count, max_count + 1):
        
        data = inv[size]
        q = percentile(data, [0, 25, 50, 75, 100])
        all_q = list([str(float(thing)) for thing in q])
#         print("\t".join(all_q))
        print 1.0 * sum(data) / len(data)


In [ ]:
def mean(data):
    if len(data):
        return 1.0 * sum(data) / len(data)
    else:
        return "N/A"

In [ ]:
econ_by_value_yes = {}
econ_by_value_no = {}
for l in langs_pruned:
    sounds = langs_pruned[l]
    number = len(sounds)
    if number not in econ_by_value_yes:
        econ_by_value_yes[number] = []
    if number not in econ_by_value_no:
        econ_by_value_no[number] = []
    econ = get_econ(sounds)
    if 'long' in sounds[0]:
        econ_by_value_yes[number].append(econ)
    else:
        econ_by_value_no[number].append(econ)


In [ ]:
import scipy.stats

In [ ]:
sizerange = range(min_count, max_count + 1)

In [ ]:
for i in sizerange:
    print scipy.stats.ttest_ind(econ_by_value[i], econ_samples[i]).pvalue

In [ ]:
def differ_by_one(v1, v2):
    counter = 0
    for key in v1:
        if key not in v2:
            print "keyerror"
            raise
        value1 = v1[key]
        value2 = v2[key]
        if value1 != value2:
            counter += 1
    if counter == 1:
        return True
    else:
        return False

def get_sym(sounds):
    num = len(sounds)
    counter = 0
    for i in range(num):
        for j in range(i + 1, num):
            if differ_by_one(sounds[i], sounds[j]):
                counter += 1
    return counter

In [ ]:
trial = langs_pruned['kir']
print trial
print get_sym(trial)

In [ ]:
sym_by_value = {}
for l in langs_pruned:
    sounds = langs_pruned[l]
    number = len(sounds)
    if number not in sym_by_value:
        sym_by_value[number] = []
    econ = get_sym(sounds)
    sym_by_value[number].append(econ)

sym_samples = {}
for size in range(min_count, max_count + 1):
    sym_samples[size] = []
    for inv in samples[size][:500]:
        econ = get_sym(inv)
        sym_samples[size].append(econ)

In [ ]:
sym_inv = [sym_by_value, sym_samples]
for inv in sym_inv:
    print("Next")
    for size in range(min_count, max_count + 1):
        data = inv[size]
        q = percentile(data, [0, 25, 50, 75, 100])
        all_q = list([str(float(thing)) for thing in q])
        print("\t".join(all_q))
#         print 1.0 * sum(data) / len(data)


In [ ]:
for i in sizerange:
    print scipy.stats.ttest_ind(sym_by_value[i], sym_samples[i]).pvalue

In [ ]:
sym_by_value_yes = {}
sym_by_value_no = {}
for l in langs_pruned:
    sounds = langs_pruned[l]
    number = len(sounds)
    if number not in sym_by_value_yes:
        sym_by_value_yes[number] = []
    if number not in sym_by_value_no:
        sym_by_value_no[number] = []
    econ = get_econ(sounds)
    if 'round' in sounds[0]:
        sym_by_value_yes[number].append(econ)
    else:
        sym_by_value_no[number].append(econ)
for i in sizerange:
    print scipy.stats.ttest_ind(sym_by_value_yes[i], sym_by_value_no[i])